In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import re
from tqdm.notebook import trange, tqdm
from tqdm.gui import tqdm as tqdm_gui
from IPython.display import display
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
dft = pd.read_csv('csv/tiraet_final_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv')

In [3]:
dft.head(2)

,url,id,_category,_subcategory,_group,brend,title,date,price,old_price,action_sale,sale,group_prise,dif_price,availability,count_group,activ_count_group,error_group,tir
0,https://tiraet.com/catalog/avtojelektronika/av...,78604,T_автотовары,T_автомагнитола,T_автомагнитола,sencor,"Автомагнитола Sencor SCT 3018MR (FM/AM, CD, MP...",2024-06-27,660.0,NaN,0,NaN,бюджет,3.331818,Мало,6,6,0,1
1,https://tiraet.com/catalog/avtojelektronika/av...,78235,T_автотовары,T_автомагнитола,T_автомагнитола,kenwood,"Автомагнитола Kenwood KDC-130UG (FM/AM, MP3/WA...",2024-06-27,1419.0,NaN,0,NaN,стандарт,3.331818,Мало,6,6,0,1


In [4]:
dfh = pd.read_csv('csv/hi_tech_final_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv')

In [5]:
dfh.head(2)

,url,id,_category,_subcategory,_group,brend,title,date,price,old_price,action_sale,sale,group_prise,dif_price,availability,count_group,activ_count_group,error_group,tir
0,https://hi-tech.md/bytovaya-tehnika/klimatiche...,Т-000078544,H_акция,H_рассрочка_12_месяцев___без_переплаты!,H_рассрочка_12_месяцев___без_переплаты!,electrolux,кондиционер Electrolux EACS-07HAT/N3_21Y,2024-06-27,3999.0,NaN,0,NaN,стандарт,1.000000,В наличии,1,1,0,0
1,https://hi-tech.md/bytovaya-tehnika/aksessuary...,Т-000056675,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,panasonic,насадка д/Зубная щетка Panasonic EW0940W830 2шт,2024-06-27,34.0,68.0,1,50.0,бюджет,2.764706,В наличии,7,7,0,0


In [6]:
df = pd.concat([dfh, dft], ignore_index=True)

In [7]:
df = df.drop_duplicates().reset_index(drop=True)

In [8]:
df.to_csv('csv/hay_tir_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv', index=False)  

In [9]:
def preprocessing(x):
    print(f'таблица имеет следующий вид:')
    display(x.head())
    print('*'*100)
    print(f'таблица имеет {x.shape[0]} сторок и {x.shape[1]} столбцов')
    print('*'*100)
    print(f'в таблице обнаружено дублекатов:{x.duplicated().sum()}')
    print('*'*100)
    print('в таблице обнаружены пропуски в следующих столбцах:')
    display(x.isna().sum())
    print('*'*100)
    print('Эти пропуски составлябт следующее количество в процентах')
    display((x.isna().mean()* 100).round(1))
    print('*'*100)
    print('столбцы имеют следующие типы:')
    display(x.dtypes)
    x.columns = [y.lower().replace(' ', '_') for y in x.columns.values] 
    print('*'*100)
    print('названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание')
    print('*'*100)
    display(x.info())

In [10]:
preprocessing(df)

таблица имеет следующий вид:


,url,id,_category,_subcategory,_group,brend,title,date,price,old_price,action_sale,sale,group_prise,dif_price,availability,count_group,activ_count_group,error_group,tir
0,https://hi-tech.md/bytovaya-tehnika/klimatiche...,Т-000078544,H_акция,H_рассрочка_12_месяцев___без_переплаты!,H_рассрочка_12_месяцев___без_переплаты!,electrolux,кондиционер Electrolux EACS-07HAT/N3_21Y,2024-06-27,3999.0,NaN,0,NaN,стандарт,1.000000,В наличии,1,1,0,0
1,https://hi-tech.md/bytovaya-tehnika/aksessuary...,Т-000056675,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,panasonic,насадка д/Зубная щетка Panasonic EW0940W830 2шт,2024-06-27,34.0,68.0,1,50.0,бюджет,2.764706,В наличии,7,7,0,0
2,https://hi-tech.md/bytovaya-tehnika/aksessuary...,Т-000021198,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,braun,насадка д/Зубная щетка Braun EB10 Kids Frozen ...,2024-06-27,94.0,NaN,0,NaN,элитный,2.764706,В наличии,7,7,0,0
3,https://hi-tech.md/bytovaya-tehnika/aksessuary...,Т-000068433,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,braun,насадка д/Зубная щетка Braun EB18 3D White (1шт),2024-06-27,94.0,NaN,0,NaN,элитный,2.764706,В наличии,7,7,0,0
4,https://hi-tech.md/bytovaya-tehnika/aksessuary...,Т-000068766,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,braun,насадка д/Зубная щетка Braun EB20 Precision Cl...,2024-06-27,94.0,NaN,0,NaN,элитный,2.764706,В наличии,7,7,0,0


****************************************************************************************************
таблица имеет 20390 сторок и 19 столбцов
****************************************************************************************************
в таблице обнаружено дублекатов:0
****************************************************************************************************
в таблице обнаружены пропуски в следующих столбцах:


url                      0
id                       0
_category                0
_subcategory             0
_group                   0
brend                    0
title                    0
date                     0
price                    2
old_price            16656
action_sale              0
sale                 16656
group_prise              2
dif_price               28
availability             0
count_group              0
activ_count_group        0
error_group              0
tir                      0
dtype: int64

****************************************************************************************************
Эти пропуски составлябт следующее количество в процентах


url                   0.0
id                    0.0
_category             0.0
_subcategory          0.0
_group                0.0
brend                 0.0
title                 0.0
date                  0.0
price                 0.0
old_price            81.7
action_sale           0.0
sale                 81.7
group_prise           0.0
dif_price             0.1
availability          0.0
count_group           0.0
activ_count_group     0.0
error_group           0.0
tir                   0.0
dtype: float64

****************************************************************************************************
столбцы имеют следующие типы:


url                   object
id                    object
_category             object
_subcategory          object
_group                object
brend                 object
title                 object
date                  object
price                float64
old_price            float64
action_sale            int64
sale                 float64
group_prise           object
dif_price            float64
availability          object
count_group            int64
activ_count_group      int64
error_group            int64
tir                    int64
dtype: object

****************************************************************************************************
названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание
****************************************************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20390 entries, 0 to 20389
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   url                20390 non-null  object 
 1   id                 20390 non-null  object 
 2   _category          20390 non-null  object 
 3   _subcategory       20390 non-null  object 
 4   _group             20390 non-null  object 
 5   brend              20390 non-null  object 
 6   title              20390 non-null  object 
 7   date               20390 non-null  object 
 8   price              20388 non-null  float64
 9   old_price          3734 non-null   float64
 10  action_sale        

None

In [11]:
df.groupby('id')['brend'].count().sort_values(ascending=False)

id
36716          2
21089          2
Т-000063284    1
Т-000063279    1
Т-000063278    1
              ..
78425          1
78423          1
78421          1
78420          1
У-000028555    1
Name: brend, Length: 20388, dtype: int64

In [12]:
df[df['id']==21089]

,url,id,_category,_subcategory,_group,brend,title,date,price,old_price,action_sale,sale,group_prise,dif_price,availability,count_group,activ_count_group,error_group,tir
12106,https://tiraet.com/catalog/Komplektujuschie/ak...,21089,T_компьютерная_техника,T_аксессуары,T_диски_дискеты,verbatim,"Диск DVD+R VERBATIM 8,5GB 8X DOUBLE LAYER PRIN...",2024-06-27,742.5,NaN,0,NaN,элитный,114.30303,Достаточно,23,23,0,1
12107,https://tiraet.com/catalog/Komplektujuschie/ak...,21089,T_компьютерная_техника,T_аксессуары,T_диски_дискеты,verbatim,"Диск DVD+R VERBATIM 8,5GB 8X DOUBLE LAYER PRIN...",2024-06-27,754.4,NaN,0,NaN,элитный,114.30303,Достаточно,23,23,0,1


In [13]:
df[df['_group']=='H_климатическая_техника']

,url,id,_category,_subcategory,_group,brend,title,date,price,old_price,action_sale,sale,group_prise,dif_price,availability,count_group,activ_count_group,error_group,tir
